### Setup

In [57]:
import os, sys
sys.path.append('..')

from cobra import io
from cobra.core import Model, Reaction, Gene, GPR
from cobra.manipulation import remove_genes
from scripts.helpers.model import rxn_in_model, gene_in_model
# Load models

# Load wildtype from manual directory (adjust path for notebooks directory)
# Use io.read_sbml_model for local files instead of io.load_model
wildtype = io.read_sbml_model('../data/fill/xmls/MNL_iCre1355_auto_GAPFILL.xml')

models = {
    "Wildtype": wildtype
}

# Get altered reactions of wildtype - using correct directory name
altered_dir = '../data/altered/xmls/MNL_iCre1355_auto_GAPFILL/'

for root, dirs, files in os.walk(altered_dir):

    # Exclude the /h directory from search
    if root.endswith('/h'):
        continue

    for file in files:
        if file.endswith('.xml'):
            # Get file name and remove .xml extension
            model_name = file[:-4]
            full_path = os.path.join(root, file)
            print(f"Loading model: {model_name} from {full_path}")
            models[model_name] = io.read_sbml_model(full_path)

No objective coefficients in model. Unclear what should be optimized


Loading model: SQE+MVA from ../data/altered/xmls/MNL_iCre1355_auto_GAPFILL/SQE+MVA.xml


No objective coefficients in model. Unclear what should be optimized


Loading model: SQE from ../data/altered/xmls/MNL_iCre1355_auto_GAPFILL/SQE.xml


No objective coefficients in model. Unclear what should be optimized


...

No objective coefficients in model. Unclear what should be optimized


Loading model: SQS+SQE from ../data/altered/xmls/MNL_iCre1355_auto_GAPFILL/SQS+SQE.xml


No objective coefficients in model. Unclear what should be optimized


Loading model: SQS from ../data/altered/xmls/MNL_iCre1355_auto_GAPFILL/SQS.xml


No objective coefficients in model. Unclear what should be optimized


### Uniprot Gene IDs

In [58]:
"""
The following reactions need to have gene associations updated to UniProt IDs for GECKO Kcat values:
    - ALT_MVAS, ALT_MVAD, ALT_MVAE, ALT_MVD, ALT_MVK, ALT_PMK, ALT_IDLI (MVA pathway)
    - SS, SMO, ERG5, ERG3, ERG (Sterol pathway)
    - ALT_PSPPS2, ALT_SQS2, ALT_SQE2 (Overexpressions)
    - DXS, CMK, CMS, HDS (MEP pathway)
    - Additional SQS, SQE strains with their kcat values

Concepts to keep in mind:
    - One gene may regulate multiple reactions
    - One reaction may be regulated by the conjunction or disjunction of multiple genes

Ensure:
    - Did we cover ever necessary reaction for proper constraining?
"""
reac_gene = {
    # MVA Pathway -- Note: what strain are we using for MVA?
    'ALT_MVAS': ['MVAS'],
    'ALT_MVAD': ['MVAD'],
    'ALT_MVAE': ['MVAE'],
    'ALT_MVK': ['MVK'],
    'ALT_PMK': ['PMK'],
    'ALT_IDLI': ['IDLI'],
    # MEP Pathway
    'DXS': ['Cre07.g356350.t1.1'],
    'CMK': ['Cre02.g145050.t1.2'],
    'CMS': ['Cre16.g679669.t1.1'],
    'HDS': ['Cre12.g490350.t1.1'],
    # Sterol Pathway
    'SS': ['Cre03.g175250.t1.2', 'Cre03.g175250.t2.1'],
    'SMO': ['Cre17.g734644.t1.1'],
    'ERG5': ['ERG5'],
    'ERG3': ['ERG3'],
    'ERG': ['ERG4'],
    # Overexpressions
    'ALT_PSPPS2': ['SQS2'],
    'ALT_SQS2': ['SQS2'],
    'ALT_SQE2': ['SQE2'],
}

gene_uniprot = {
    # MVA Pathway
    'MVAS': 'Q9FD71', # https://www.uniprot.org/uniprotkb/Q9FD71/entry
    'MVAE': 'Q9FD65', # https://www.uniprot.org/uniprotkb/Q9FD65/publications
    'MVK': 'Q8PW39', # https://www.uniprot.org/uniprotkb/Q8PW39/entry
    'PMK': 'Q04430', # https://www.uniprot.org/uniprotkb/Q04430/entry
    'MVAD': 'P32377', # https://www.uniprot.org/uniprotkb/P32377/entry
    'IDLI': '',
    # MEP Pathway
    'Cre07.g356350.t1.1': 'O81954', # https://www.uniprot.org/uniprotkb/O81954/entry
    'Cre02.g145050.t1.2': 'O81014', # https://www.uniprot.org/uniprotkb/O81014/entry (assumes CMK/ISPE of Arabidopsis thaliana)
    'Cre16.g679669.t1.1': '',
    'Cre12.g490350.t1.1': '',
    # Sterol Pathway
    'Cre03.g175250.t1.2': 'B4DWP0', # https://www.uniprot.org/uniprotkb/B4DWP0/entry (assumes sqs of homo sapiens)
    'Cre03.g175250.t2.1': 'B4DWP0', # ... (likewise)
    'Cre17.g734644.t1.1': 'P52020', # https://www.uniprot.org/uniprotkb/P52020/publications (assumes sqe of rattus norvegicus)
    # 'ERG3': '',
    # 'ERG5': '',
    # 'ERG4': '',
    # Overexpressions (these will be storing an object with different gene id for strains)
    'SQS2': {

    },
    'SQE2': {

    },
}

# Valid UniProt mappings with example kcat values (1/s)
kcat_database = {
    'Q9FD71': 5.2,   # MVAS - 3-hydroxy-3-methylglutaryl-CoA synthase
    'Q9FD65': 3.8,   # MVAE - Mevalonate kinase
    'O81954': 12.5,  # DXS - 1-deoxy-D-xylulose-5-phosphate synthase
    'O81014': 8.9,   # CMK - 4-diphosphocytidyl-2-C-methyl-D-erythritol kinase
    'B4DWP0': 15.3,  # SS - Squalene synthase
    'P52020': 7.2,   # SMO - Sterol 14α-demethylase
}
    
# Molecular weights (kDa) - approximate values for enzyme constraint calculations
molecular_weights = {
    'Q9FD71': 57.0,  # MVAS
    'Q9FD65': 42.0,  # MVAE
    'O81954': 67.0,  # DXS
    'O81014': 32.0,  # CMK
    'B4DWP0': 48.0,  # SS
    'P52020': 52.0,  # SMO
}

# Enzyme availability (mmol enzyme/gDW) - realistic cellular concentrations
enzyme_availability = {
    'Q9FD71': 0.001,   # MVAS
    'Q9FD65': 0.001,   # MVAE  
    'O81954': 0.002,   # DXS
    'O81014': 0.001,   # CMK
    'B4DWP0': 0.001,   # SS
    'P52020': 0.0005,  # SMO
}

# Protein availability (assuming 1 mmol/gDW total protein pool per enzyme)
protein_availability = {
    'Q9FD71': 1.0,   # MVAS
    'Q9FD65': 1.0,   # MVAE  
    'O81954': 1.0,   # DXS
    'O81014': 1.0,   # CMK
    'B4DWP0': 1.0,   # SS
    'P52020': 1.0,   # SMO
}
    

The following script rebuilds the model gene associations where the above stated gene ids are replaced with the uniprot equivalent:

In [59]:
def rxns_with_gene(m: Model, gene_id: str) -> list[Reaction]:

    return [rxn for rxn in m.reactions if gene_id in list(map(lambda x: x.id, rxn.genes))]

def rebuild_model(m: Model) -> Model:

    new_model: Model = m.copy()

    removal: list[Gene] = []
    
    for gene in new_model.genes:

        if gene.id not in gene_uniprot.keys(): continue

        # Special handling for this case
        if gene.id in ['SQS2', 'SQE2']:
            continue
        
        old_gid = gene.id
        new_gid = gene_uniprot[old_gid]
        
        # If new_gid is empty, skip
        if new_gid == "": continue

        # If new_gid already in model, mark current gene for removal and skip
        if gene_in_model(new_model, new_gid):
            removal.append(gene)
            continue
        
        gene.id = new_gid

        print(f"New gene: {new_gid} (old: {old_gid})")
        # Find all reactions that are associated with the gene and update the gpr / gene_reaction
        rxns = rxns_with_gene(new_model, gene.id)
        for rxn in rxns:
            print(f"\t{rxn.id} --- {rxn.gene_reaction_rule}")
            if old_gid in rxn.gene_reaction_rule:
                rxn.gene_reaction_rule = rxn.gene_reaction_rule.replace(old_gid, new_gid)
                if rxn.gene_reaction_rule.count(new_gid) > 1:
                    rxn.gene_reaction_rule = new_gid
                # rxn.gene_reaction_rulue = new_gid

    for rgene in removal:
        print(f"Removing duplicate gene: {rgene.id}")
        remove_genes(new_model, [rgene], remove_reactions=False)
    print('\n')
    
    return new_model

# Models updated with UniProt IDs
updated_models = {k: rebuild_model(v) for k, v in models.items()}

New gene: P52020 (old: Cre17.g734644.t1.1)
	SMO --- Cre17.g734644.t1.1
New gene: O81014 (old: Cre02.g145050.t1.2)
	CMK --- Cre02.g145050.t1.2
New gene: O81954 (old: Cre07.g356350.t1.1)
	DXS --- Cre07.g356350.t1.1
New gene: B4DWP0 (old: Cre03.g175250.t2.1)
	PSPPS --- Cre03.g175250.t1.2 or Cre03.g175250.t2.1
	SS --- Cre03.g175250.t1.2 or Cre03.g175250.t2.1
Removing duplicate gene: Cre03.g175250.t1.2


New gene: P52020 (old: Cre17.g734644.t1.1)
	SMO --- Cre17.g734644.t1.1
New gene: O81014 (old: Cre02.g145050.t1.2)
	CMK --- Cre02.g145050.t1.2
New gene: O81954 (old: Cre07.g356350.t1.1)
	DXS --- Cre07.g356350.t1.1
New gene: B4DWP0 (old: Cre03.g175250.t2.1)
	PSPPS --- Cre03.g175250.t1.2 or Cre03.g175250.t2.1
	SS --- Cre03.g175250.t1.2 or Cre03.g175250.t2.1
New gene: Q9FD71 (old: MVAS)
	ALT_MVAS --- MVAS
New gene: Q9FD65 (old: MVAE)
	ALT_MVAE --- MVAE
New gene: Q04430 (old: PMK)
	ALT_PMK --- PMK
New gene: Q8PW39 (old: MVK)
	ALT_MVK --- MVK
New gene: P32377 (old: MVAD)
	ALT_MVAD --- MVAD
Removi

In [60]:
def remove_duplicate_genes(model: Model) -> Model:
    """
    Remove duplicate genes (genes with the same ID) from a model.
    Keeps the first occurrence and removes subsequent duplicates.
    """
    # First, identify duplicates in the original model
    seen_gene_ids = set()
    genes_to_remove = []
    
    for gene in model.genes:
        if gene.id in seen_gene_ids:
            print(f"Found duplicate gene: {gene.id}")
            genes_to_remove.append(gene)
        else:
            seen_gene_ids.add(gene.id)
    
    # If no duplicates found, just return a copy
    if not genes_to_remove:
        print("No duplicate genes found")
        return model.copy()
    
    # Remove duplicate genes from the original model (in place)
    for gene in genes_to_remove:
        print(f"Removing duplicate gene: {gene.id}")
        remove_genes(model, [gene], remove_reactions=False)
    
    # Now we can safely copy the model
    return model.copy()

# Clean all models by removing duplicate genes
print("Removing duplicate genes from all models...")
cleaned_models = {}
for k, m in updated_models.items():
    print(f"\nCleaning model: {k}")
    cleaned_models[k] = remove_duplicate_genes(m)
    
    # Verify no duplicates remain
    gene_counts = {}
    for gene in cleaned_models[k].genes:
        gene_counts[gene.id] = gene_counts.get(gene.id, 0) + 1
    
    duplicates = {gid: count for gid, count in gene_counts.items() if count > 1}
    if duplicates:
        print(f"  WARNING: Still has duplicates: {duplicates}")
    else:
        print(f"  Clean: {len(cleaned_models[k].genes)} unique genes")

# Update the models dictionary
updated_models = cleaned_models

Removing duplicate genes from all models...

Cleaning model: Wildtype
Found duplicate gene: P52020
Found duplicate gene: O81014
Found duplicate gene: O81954
Found duplicate gene: B4DWP0
Removing duplicate gene: P52020
Removing duplicate gene: O81014
Removing duplicate gene: O81954
Removing duplicate gene: B4DWP0
  Clean: 1967 unique genes

Cleaning model: SQE+MVA
Found duplicate gene: P52020
Found duplicate gene: O81014
Found duplicate gene: O81954
Found duplicate gene: B4DWP0
Found duplicate gene: Q9FD71
Found duplicate gene: Q9FD65
Found duplicate gene: Q04430
Found duplicate gene: Q8PW39
Found duplicate gene: P32377
Removing duplicate gene: P52020
Removing duplicate gene: O81014
Removing duplicate gene: O81954
Removing duplicate gene: B4DWP0
Removing duplicate gene: Q9FD71
Removing duplicate gene: Q9FD65
Removing duplicate gene: Q04430
Removing duplicate gene: Q8PW39
Removing duplicate gene: P32377
  Clean: 1974 unique genes

Cleaning model: SQE
Found duplicate gene: P52020
Found du

### Testing New Models

In [61]:
for k, m in updated_models.items():

    print(f"Model: {k}")

    for rid in reac_gene.keys():

        if not rxn_in_model(m, rid): continue

        rxn = m.reactions.get_by_id(rid)

        print(f"\tReaction: {rid}\n\t\tGPR:{rxn.gpr}\n\t\tGenes:{rxn.genes}\n\t\tGene-Reaction:{rxn.gene_reaction_rule}\n")

Model: Wildtype
	Reaction: DXS
		GPR:
		Genes:frozenset()
		Gene-Reaction:

	Reaction: CMK
		GPR:
		Genes:frozenset()
		Gene-Reaction:

	Reaction: CMS
		GPR:Cre16.g679669.t1.1
		Genes:frozenset({<Gene Cre16.g679669.t1.1 at 0x1a50bb8fbc0>})
		Gene-Reaction:Cre16.g679669.t1.1

	Reaction: HDS
		GPR:Cre12.g490350.t1.1
		Genes:frozenset({<Gene Cre12.g490350.t1.1 at 0x1a50bb8fcb0>})
		Gene-Reaction:Cre12.g490350.t1.1

	Reaction: SS
		GPR:
		Genes:frozenset()
		Gene-Reaction:

	Reaction: SMO
		GPR:
		Genes:frozenset()
		Gene-Reaction:

	Reaction: ERG5
		GPR:ERG5
		Genes:frozenset({<Gene ERG5 at 0x1a501bb2120>})
		Gene-Reaction:ERG5

	Reaction: ERG3
		GPR:ERG3
		Genes:frozenset({<Gene ERG3 at 0x1a501bb20f0>})
		Gene-Reaction:ERG3

	Reaction: ERG
		GPR:ERG4
		Genes:frozenset({<Gene ERG4 at 0x1a501bb2150>})
		Gene-Reaction:ERG4

Model: SQE+MVA
	Reaction: ALT_MVAS
		GPR:
		Genes:frozenset()
		Gene-Reaction:

	Reaction: ALT_MVAD
		GPR:
		Genes:frozenset()
		Gene-Reaction:

	Reaction: ALT_MVAE
		GP

In [62]:
def get_gene_count():    
    for k, m in updated_models.items():
        print(f"Model: {k}")
        for gold, gnew in gene_uniprot.items():
            if gnew == "" or gold in ['SQS2', 'SQE2']: continue
            print(f"\t{gnew} --- {len([g for g in m.genes if g.id == gnew])}")

get_gene_count()

Model: Wildtype
	Q9FD71 --- 0
	Q9FD65 --- 0
	Q8PW39 --- 0
	Q04430 --- 0
	P32377 --- 0
	O81954 --- 1
	O81014 --- 1
	B4DWP0 --- 1
	B4DWP0 --- 1
	P52020 --- 1
Model: SQE+MVA
	Q9FD71 --- 1
	Q9FD65 --- 1
	Q8PW39 --- 1
	Q04430 --- 1
	P32377 --- 1
	O81954 --- 1
	O81014 --- 1
	B4DWP0 --- 1
	B4DWP0 --- 1
	P52020 --- 1
Model: SQE
	Q9FD71 --- 0
	Q9FD65 --- 0
	Q8PW39 --- 0
	Q04430 --- 0
	P32377 --- 0
	O81954 --- 1
	O81014 --- 1
	B4DWP0 --- 1
	B4DWP0 --- 1
	P52020 --- 1
Model: SQS+MVA
	Q9FD71 --- 1
	Q9FD65 --- 1
	Q8PW39 --- 1
	Q04430 --- 1
	P32377 --- 1
	O81954 --- 1
	O81014 --- 1
	B4DWP0 --- 1
	B4DWP0 --- 1
	P52020 --- 1
Model: SQS+SQE+MVA
	Q9FD71 --- 1
	Q9FD65 --- 1
	Q8PW39 --- 1
	Q04430 --- 1
	P32377 --- 1
	O81954 --- 1
	O81014 --- 1
	B4DWP0 --- 1
	B4DWP0 --- 1
	P52020 --- 1
Model: SQS+SQE
	Q9FD71 --- 0
	Q9FD65 --- 0
	Q8PW39 --- 0
	Q04430 --- 0
	P32377 --- 0
	O81954 --- 1
	O81014 --- 1
	B4DWP0 --- 1
	B4DWP0 --- 1
	P52020 --- 1
Model: SQS
	Q9FD71 --- 0
	Q9FD65 --- 0
	Q8PW39 --- 0
	Q04430 --- 0
	P

### Save Models

In [63]:
from scripts.helpers.model import add_single_gene_reaction_pair, met_in_model

save_path = os.path.join('..', 'data', 'gecko', 'prev', 'xmls')
os.makedirs(save_path, exist_ok=True)
for k, m in updated_models.items():

    # Add ergosterol & orthophosphate sink reactions if not present
    # Add ergosterol exchange reaction
    ERG = "ergosterol_c"
    ERGEXCH = "ERGOSTEROLEXCH"
    if not rxn_in_model(m, ERGEXCH):
        add_single_gene_reaction_pair(
            model=m, 
            gene_id="EXCHERG_GENE",
            reaction_id=ERGEXCH,
            reaction_name="Ergosterol exchange (assumption)", 
            reaction_subsystem="Exchange", 
            metabolites=[(-1, ERG)],
            reversible=True
        )

    ORTHOP = "orthop_c"
    EXCHORTHOP = "ORTHOPHOSPHATEEXCH"
    if not rxn_in_model(m, EXCHORTHOP) and met_in_model(m, ORTHOP):
        add_single_gene_reaction_pair(
            model=m,
            gene_id="EXCHORTHOP",
            reaction_id=EXCHORTHOP,
            reaction_name="Orthophosphate exchange (assumption)",
            reaction_subsystem="Exchange",
            metabolites=[(-1, ORTHOP)],
            reversible=True
        )
    
    io.write_sbml_model(m, os.path.join(save_path, f"{k}_updated.xml"))

### Build GECKO Model

In [64]:
from geckopy.gecko import GeckoModel
from cobra import Reaction, Metabolite

# Function to add required GECKO pool exchange reactions
def add_gecko_pool_exchanges(model):
    """Add the pool exchange reactions that GECKO expects"""
    gecko_pool_reactions = {
        'r_4041': 'protein pool exchange',
        'r_4047': 'carbohydrate pool exchange', 
        'r_4048': 'lipid pool exchange',
        'r_4049': 'cofactor pool exchange',
        'r_4050': 'nucleotide pool exchange'
    }
    
    added_reactions = []
    for rxn_id, description in gecko_pool_reactions.items():
        if rxn_id not in [r.id for r in model.reactions]:
            # Create pool exchange reaction
            pool_rxn = Reaction(rxn_id)
            pool_rxn.name = f"GECKO {description}"
            
            # Create pool metabolite
            pool_met = Metabolite(f"pool_met_{rxn_id}")
            pool_met.name = f"Pool metabolite for {description}"
            pool_met.compartment = "c"
            
            # Set up as exchange reaction (pool metabolite -> )
            pool_rxn.add_metabolites({pool_met: -1})
            pool_rxn.bounds = (0, 1000)  # Allow consumption from pool
            
            model.add_reactions([pool_rxn])
            added_reactions.append(rxn_id)
    
    return added_reactions

# run geckopy.io.read_sbml_model on all models in ./data/gecko/prev/xmls
gecko_models = {}
gecko_dir = os.path.join('..', 'data', 'gecko', 'prev', 'xmls')
for root, dirs, files in os.walk(gecko_dir):
    for file in files:
        if file.endswith('.xml'):
            model_name = file[:-4]
            full_path = os.path.join(root, file)
            print(f"Loading GECKO model: {model_name} from {full_path}")
            try:
                cm = io.read_sbml_model(full_path)
                
                # Add required GECKO pool exchange reactions
                added_rxns = add_gecko_pool_exchanges(cm)
                if added_rxns:
                    print(f"  Added GECKO pool exchanges: {added_rxns}")
                
                # Check biomass reactions
                biomass_reactions = [r.id for r in cm.reactions if 'biomass' in r.id.lower()]
                print(f"  Found biomass reactions: {biomass_reactions}")
                
                # Create GECKO model
                gecko_models[model_name] = GeckoModel(cm)
                print(f"✓ Successfully created GECKO model: {model_name}")
                print(f"  GECKO model has {len(gecko_models[model_name].reactions)} reactions")
                    
            except Exception as e:
                print(f"✗ Failed to create GECKO model {model_name}: {str(e)}")
                continue

Loading GECKO model: SQE+MVA_updated from ..\data\gecko\prev\xmls\SQE+MVA_updated.xml


No objective coefficients in model. Unclear what should be optimized


  Added GECKO pool exchanges: ['r_4041', 'r_4047', 'r_4048', 'r_4049', 'r_4050']
  Found biomass reactions: ['Biomass_Chlamy_auto', 'Biomass_Chlamy_mixo', 'Biomass_Chlamy_hetero']
✓ Successfully created GECKO model: SQE+MVA_updated
  GECKO model has 2414 reactions
Loading GECKO model: SQE_updated from ..\data\gecko\prev\xmls\SQE_updated.xml


No objective coefficients in model. Unclear what should be optimized


  Added GECKO pool exchanges: ['r_4041', 'r_4047', 'r_4048', 'r_4049', 'r_4050']
  Found biomass reactions: ['Biomass_Chlamy_auto', 'Biomass_Chlamy_mixo', 'Biomass_Chlamy_hetero']
✓ Successfully created GECKO model: SQE_updated
  GECKO model has 2408 reactions
Loading GECKO model: SQS+MVA_updated from ..\data\gecko\prev\xmls\SQS+MVA_updated.xml


...

  Added GECKO pool exchanges: ['r_4041', 'r_4047', 'r_4048', 'r_4049', 'r_4050']
  Found biomass reactions: ['Biomass_Chlamy_auto', 'Biomass_Chlamy_mixo', 'Biomass_Chlamy_hetero']
✓ Successfully created GECKO model: SQS+SQE_updated
  GECKO model has 2410 reactions
Loading GECKO model: SQS_updated from ..\data\gecko\prev\xmls\SQS_updated.xml


No objective coefficients in model. Unclear what should be optimized


  Added GECKO pool exchanges: ['r_4041', 'r_4047', 'r_4048', 'r_4049', 'r_4050']
  Found biomass reactions: ['Biomass_Chlamy_auto', 'Biomass_Chlamy_mixo', 'Biomass_Chlamy_hetero']
✓ Successfully created GECKO model: SQS_updated
  GECKO model has 2409 reactions
Loading GECKO model: Wildtype_updated from ..\data\gecko\prev\xmls\Wildtype_updated.xml


No objective coefficients in model. Unclear what should be optimized


  Added GECKO pool exchanges: ['r_4041', 'r_4047', 'r_4048', 'r_4049', 'r_4050']
  Found biomass reactions: ['Biomass_Chlamy_auto', 'Biomass_Chlamy_mixo', 'Biomass_Chlamy_hetero']
✓ Successfully created GECKO model: Wildtype_updated
  GECKO model has 2406 reactions


### Gap-fill Missing Kcat

In [65]:
# Add kcat constraints to GECKO models using UniProt IDs
# This integrates enzyme kinetics data to constrain metabolic fluxes
from cobra import Reaction, Metabolite

def add_manual_kcat_constraints(gecko_models_dict, reac_gene_dict, gene_uniprot_dict):
    """
    Manually add kcat constraints to GECKO models using protein exchange reactions
    """
    print("=== MANUAL KCAT CONSTRAINT IMPLEMENTATION ===\n")
    
    valid_gene_mappings = {
        old_gene: new_gene for old_gene, new_gene in gene_uniprot_dict.items()
        if new_gene != "" and old_gene not in ['SQS2', 'SQE2'] and not isinstance(new_gene, dict)
    }
    
    print("Available kcat values:")
    for uniprot_id, kcat in kcat_database.items():
        print(f"  {uniprot_id}: {kcat} s⁻¹")
    print()
    
    constraints_added = 0
    
    for model_name, gecko_model in gecko_models_dict.items():
        print(f"Adding kcat constraints to model: {model_name}")
        
        model_constraints = 0
        
        # Process each target reaction
        for reaction_id, associated_genes in reac_gene_dict.items():
            if reaction_id not in [r.id for r in gecko_model.reactions]:
                continue
                
            # Get UniProt IDs for this reaction
            uniprot_genes = [valid_gene_mappings.get(gene) for gene in associated_genes if gene in valid_gene_mappings]
            uniprot_genes = [g for g in uniprot_genes if g is not None and g in kcat_database]
            
            if not uniprot_genes:
                continue
                
            reaction = gecko_model.reactions.get_by_id(reaction_id)
            print(f"  Processing {reaction_id} with proteins: {uniprot_genes}")
            
            for uniprot_id in uniprot_genes:
                try:
                    # Method 1: Create enzyme usage reaction
                    enzyme_rxn_id = f"usage_{uniprot_id}_{reaction_id}"
                    
                    # Check if enzyme usage reaction already exists
                    if enzyme_rxn_id not in [r.id for r in gecko_model.reactions]:
                        # Create enzyme usage reaction
                        enzyme_rxn = Reaction(enzyme_rxn_id)
                        enzyme_rxn.name = f"Enzyme usage: {uniprot_id} for {reaction_id}"
                        enzyme_rxn.subsystem = "Enzyme usage"
                        
                        # Create enzyme metabolite if it doesn't exist
                        enzyme_met_id = f"prot_{uniprot_id}_c"
                        if enzyme_met_id not in [m.id for m in gecko_model.metabolites]:
                            enzyme_met = Metabolite(enzyme_met_id)
                            enzyme_met.name = f"Protein {uniprot_id}"
                            enzyme_met.compartment = "c"
                        else:
                            enzyme_met = gecko_model.metabolites.get_by_id(enzyme_met_id)
                        
                        # Set up enzyme usage stoichiometry
                        # Enzyme usage = flux / kcat
                        kcat_value = kcat_database[uniprot_id]
                        enzyme_coeff = 1.0 / kcat_value  # enzyme amount needed per unit flux
                        
                        enzyme_rxn.add_metabolites({enzyme_met: enzyme_coeff})
                        enzyme_rxn.bounds = (0, 1000)  # Constrain to positive values
                        
                        # Add the enzyme usage reaction
                        gecko_model.add_reactions([enzyme_rxn])
                        
                        print(f"    ✓ Added enzyme usage reaction: {enzyme_rxn_id}")
                        model_constraints += 1
                        
                    # Method 2: Link enzyme usage to the actual metabolic reaction
                    # This creates the coupling between metabolic flux and enzyme usage
                    coupling_rxn_id = f"coupling_{uniprot_id}_{reaction_id}"
                    
                    if coupling_rxn_id not in [r.id for r in gecko_model.reactions]:
                        coupling_rxn = Reaction(coupling_rxn_id)
                        coupling_rxn.name = f"Enzyme coupling: {reaction_id} <-> {uniprot_id}"
                        coupling_rxn.subsystem = "Enzyme coupling"
                        
                        # Get the enzyme metabolite
                        enzyme_met = gecko_model.metabolites.get_by_id(f"prot_{uniprot_id}_c")
                        
                        # Coupling constraint: metabolic flux * enzyme_coeff = enzyme usage
                        kcat_value = kcat_database[uniprot_id]
                        coupling_coeff = 1.0 / kcat_value
                        
                        coupling_rxn.add_metabolites({enzyme_met: -coupling_coeff})
                        coupling_rxn.bounds = (-1000, 1000)
                        
                        gecko_model.add_reactions([coupling_rxn])
                        print(f"    ✓ Added coupling constraint: {coupling_rxn_id}")
                        
                except Exception as e:
                    print(f"    ✗ Failed to add constraint for {uniprot_id}: {e}")
                    
        constraints_added += model_constraints
        print(f"  Added {model_constraints} enzyme constraints to {model_name}")
        print()
    
    print(f"Total enzyme constraints added: {constraints_added}")
    return constraints_added > 0

# Apply manual kcat constraints
manual_constraints_success = add_manual_kcat_constraints(gecko_models, reac_gene, gene_uniprot)

=== MANUAL KCAT CONSTRAINT IMPLEMENTATION ===

Available kcat values:
  Q9FD71: 5.2 s⁻¹
  Q9FD65: 3.8 s⁻¹
  O81954: 12.5 s⁻¹
  O81014: 8.9 s⁻¹
  B4DWP0: 15.3 s⁻¹
  P52020: 7.2 s⁻¹

Adding kcat constraints to model: SQE+MVA_updated
  Processing ALT_MVAS with proteins: ['Q9FD71']
    ✓ Added enzyme usage reaction: usage_Q9FD71_ALT_MVAS
    ✓ Added coupling constraint: coupling_Q9FD71_ALT_MVAS
  Processing ALT_MVAE with proteins: ['Q9FD65']
    ✓ Added enzyme usage reaction: usage_Q9FD65_ALT_MVAE
    ✓ Added coupling constraint: coupling_Q9FD65_ALT_MVAE
  Processing DXS with proteins: ['O81954']
    ✓ Added enzyme usage reaction: usage_O81954_DXS
    ✓ Added coupling constraint: coupling_O81954_DXS
  Processing CMK with proteins: ['O81014']
    ✓ Added enzyme usage reaction: usage_O81014_CMK
    ✓ Added coupling constraint: coupling_O81014_CMK
  Processing SS with proteins: ['B4DWP0', 'B4DWP0']
    ✓ Added enzyme usage reaction: usage_B4DWP0_SS
    ✓ Added coupling constraint: coupling_B4

In [66]:
# Verify that kcat constraints are actually in the GECKO models
print("=== VERIFYING KCAT CONSTRAINTS IN GECKO MODELS ===\n")

for model_name, gecko_model in gecko_models.items():
    print(f"Model: {model_name}")
    
    # Count enzyme-related reactions
    usage_reactions = [r for r in gecko_model.reactions if 'usage_' in r.id]
    coupling_reactions = [r for r in gecko_model.reactions if 'coupling_' in r.id]
    protein_metabolites = [m for m in gecko_model.metabolites if 'prot_' in m.id and m.id != 'prot_pool_c']
    
    print(f"  Usage reactions: {len(usage_reactions)}")
    print(f"  Coupling reactions: {len(coupling_reactions)}")
    print(f"  Protein metabolites: {len(protein_metabolites)}")
    
    # Show specific examples if they exist
    if usage_reactions:
        print(f"  Example usage reactions:")
        for rxn in usage_reactions[:3]:  # Show first 3
            print(f"    {rxn.id}: {rxn.name}")
    
    if protein_metabolites:
        print(f"  Example protein metabolites:")
        for met in protein_metabolites[:3]:  # Show first 3
            print(f"    {met.id}: {met.name}")
    
    print()

# Overall summary
total_usage = sum(len([r for r in model.reactions if 'usage_' in r.id]) for model in gecko_models.values())
total_coupling = sum(len([r for r in model.reactions if 'coupling_' in r.id]) for model in gecko_models.values())
total_proteins = sum(len([m for m in model.metabolites if 'prot_' in m.id and m.id != 'prot_pool_c']) for model in gecko_models.values())

print(f"📊 TOTAL ACROSS ALL MODELS:")
print(f"  Total usage reactions: {total_usage}")
print(f"  Total coupling reactions: {total_coupling}")
print(f"  Total protein metabolites: {total_proteins}")

if total_usage > 0:
    print(f"\n✅ SUCCESS: Kcat constraints ARE integrated into gecko_models!")
else:
    print(f"\n❌ ISSUE: No kcat constraints found in gecko_models!")

=== VERIFYING KCAT CONSTRAINTS IN GECKO MODELS ===

Model: SQE+MVA_updated
  Usage reactions: 6
  Coupling reactions: 6
  Protein metabolites: 7
  Example usage reactions:
    usage_Q9FD71_ALT_MVAS: Enzyme usage: Q9FD71 for ALT_MVAS
    usage_Q9FD65_ALT_MVAE: Enzyme usage: Q9FD65 for ALT_MVAE
    usage_O81954_DXS: Enzyme usage: O81954 for DXS
  Example protein metabolites:
    prot_pool: 
    prot_Q9FD71_c: Protein Q9FD71
    prot_Q9FD65_c: Protein Q9FD65

Model: SQE_updated
  Usage reactions: 4
  Coupling reactions: 4
  Protein metabolites: 5
  Example usage reactions:
    usage_O81954_DXS: Enzyme usage: O81954 for DXS
    usage_O81014_CMK: Enzyme usage: O81014 for CMK
    usage_B4DWP0_SS: Enzyme usage: B4DWP0 for SS
  Example protein metabolites:
    prot_pool: 
    prot_O81954_c: Protein O81954
    prot_O81014_c: Protein O81014

Model: SQS+MVA_updated
  Usage reactions: 6
  Coupling reactions: 6
  Protein metabolites: 7
  Example usage reactions:
    usage_Q9FD71_ALT_MVAS: Enzyme us

In [67]:
# Test if kcat constraints are actually working by checking flux bounds
print("=== TESTING IF KCAT CONSTRAINTS ARE ACTUALLY WORKING ===\n")

# Pick one GECKO model to test
test_model = list(gecko_models.values())[0]
test_model_name = list(gecko_models.keys())[0]

print(f"Testing model: {test_model_name}")

# Check if we have enzyme constraints for specific reactions
target_reactions = ['ALT_MVAS', 'DXS', 'SS', 'SMO']

for rxn_id in target_reactions:
    if rxn_id in [r.id for r in test_model.reactions]:
        rxn = test_model.reactions.get_by_id(rxn_id)
        print(f"\nReaction: {rxn_id}")
        print(f"  Original bounds: {rxn.bounds}")
        
        # Look for related enzyme usage reactions
        usage_rxns = [r for r in test_model.reactions if f'usage_' in r.id and rxn_id in r.id]
        coupling_rxns = [r for r in test_model.reactions if f'coupling_' in r.id and rxn_id in r.id]
        
        print(f"  Related usage reactions: {len(usage_rxns)}")
        print(f"  Related coupling reactions: {len(coupling_rxns)}")
        
        if usage_rxns:
            for usage_rxn in usage_rxns:
                print(f"    {usage_rxn.id}: {usage_rxn.reaction}")
        
        if coupling_rxns:
            for coupling_rxn in coupling_rxns:
                print(f"    {coupling_rxn.id}: {coupling_rxn.reaction}")

# Check if the constraints actually limit flux
print(f"\n=== FLUX CONSTRAINT TEST ===")
try:
    # Set an enzyme-limited reaction as objective and see if it's constrained
    if 'ALT_MVAS' in [r.id for r in test_model.reactions]:
        test_model.objective = 'ALT_MVAS'
        solution = test_model.optimize()
        print(f"ALT_MVAS flux: {solution.fluxes['ALT_MVAS']:.6f}")
        
        # Check enzyme usage
        mvas_usage = [r for r in test_model.reactions if 'usage_Q9FD71_ALT_MVAS' in r.id]
        if mvas_usage:
            usage_flux = solution.fluxes[mvas_usage[0].id]
            print(f"Enzyme usage flux: {usage_flux:.6f}")
            print(f"Kcat constraint working: {abs(usage_flux - solution.fluxes['ALT_MVAS']/5.2) < 1e-6}")
        else:
            print("No enzyme usage reaction found!")
            
except Exception as e:
    print(f"Error in flux test: {e}")

print(f"\n🔍 DIAGNOSIS:")
print(f"The enzyme reactions exist but may not be properly constraining the metabolic fluxes.")
print(f"The coupling mechanism needs to be fixed to actually link enzyme availability to reaction rates.")

=== TESTING IF KCAT CONSTRAINTS ARE ACTUALLY WORKING ===

Testing model: SQE+MVA_updated

Reaction: ALT_MVAS
  Original bounds: (0.0, 1000.0)
  Related usage reactions: 1
  Related coupling reactions: 1
    usage_Q9FD71_ALT_MVAS:  --> 0.1923076923076923 prot_Q9FD71_c
    coupling_Q9FD71_ALT_MVAS: 0.1923076923076923 prot_Q9FD71_c <=> 

Reaction: DXS
  Original bounds: (0.0, 1000.0)
  Related usage reactions: 1
  Related coupling reactions: 1
    usage_O81954_DXS:  --> 0.08 prot_O81954_c
    coupling_O81954_DXS: 0.08 prot_O81954_c <=> 

Reaction: SS
  Original bounds: (0.0, 1000.0)
  Related usage reactions: 1
  Related coupling reactions: 1
    usage_B4DWP0_SS:  --> 0.06535947712418301 prot_B4DWP0_c
    coupling_B4DWP0_SS: 0.06535947712418301 prot_B4DWP0_c <=> 

Reaction: SMO
  Original bounds: (-1000.0, 1000.0)
  Related usage reactions: 1
  Related coupling reactions: 1
    usage_P52020_SMO:  --> 0.1388888888888889 prot_P52020_c
    coupling_P52020_SMO: 0.1388888888888889 prot_P52020_

In [68]:
# PROPER KCAT CONSTRAINT IMPLEMENTATION
# This will actually constrain the GECKO models using enzyme kinetics
from cobra import Reaction, Metabolite

def add_proper_kcat_constraints(gecko_models_dict, reac_gene_dict, gene_uniprot_dict):
    """
    Add proper kcat constraints that actually limit metabolic fluxes based on enzyme availability
    """
    
    valid_gene_mappings = {
        old_gene: new_gene for old_gene, new_gene in gene_uniprot_dict.items()
        if new_gene != "" and old_gene not in ['SQS2', 'SQE2'] and not isinstance(new_gene, dict)
    }
    
    print("Kcat constraints to be applied:")
    for uniprot_id, kcat in kcat_database.items():
        max_flux = kcat * enzyme_availability[uniprot_id]
        print(f"  {uniprot_id}: kcat={kcat:.1f} s⁻¹, availability={enzyme_availability[uniprot_id]:.4f} mmol/gDW")
        print(f"    → Max flux: {max_flux:.6f} mmol/gDW/h")
    
    constraints_applied = 0
    
    for model_name, gecko_model in gecko_models_dict.items():
        print(f"Applying constraints to model: {model_name}")
        
        model_constraints = 0
        
        # Process each target reaction
        for reaction_id, associated_genes in reac_gene_dict.items():
            if reaction_id not in [r.id for r in gecko_model.reactions]:
                continue
                
            # Get UniProt IDs for this reaction
            uniprot_genes = [valid_gene_mappings.get(gene) for gene in associated_genes if gene in valid_gene_mappings]
            uniprot_genes = [g for g in uniprot_genes if g is not None and g in kcat_database]
            
            if not uniprot_genes:
                continue
                
            reaction = gecko_model.reactions.get_by_id(reaction_id)
            original_bounds = reaction.bounds
            
            print(f"  Processing {reaction_id} with enzymes: {uniprot_genes}")
            
            # Calculate flux constraint based on enzyme kinetics
            # For multiple enzymes, use the most limiting one (minimum constraint)
            max_allowed_flux = float('inf')
            limiting_enzyme = None
            
            for uniprot_id in uniprot_genes:
                kcat = kcat_database[uniprot_id]
                enzyme_conc = enzyme_availability[uniprot_id]
                max_flux = kcat * enzyme_conc  # mmol/gDW/h
                
                if max_flux < max_allowed_flux:
                    max_allowed_flux = max_flux
                    limiting_enzyme = uniprot_id
            
            # Apply the constraint to the reaction bounds
            if max_allowed_flux < float('inf'):
                # For irreversible reactions (lower bound >= 0)
                if original_bounds[0] >= 0:
                    new_upper_bound = min(original_bounds[1], max_allowed_flux)
                    reaction.bounds = (original_bounds[0], new_upper_bound)
                # For reversible reactions
                else:
                    new_upper_bound = min(original_bounds[1], max_allowed_flux)
                    new_lower_bound = max(original_bounds[0], -max_allowed_flux)
                    reaction.bounds = (new_lower_bound, new_upper_bound)
                
                print(f"    ✓ Applied constraint: {original_bounds} → {reaction.bounds}")
                print(f"      Limiting enzyme: {limiting_enzyme} (max flux: {max_allowed_flux:.6f})")
                model_constraints += 1
            
        constraints_applied += model_constraints
        print(f"  Applied {model_constraints} kcat constraints to {model_name}")
        print()
    
    print(f"Total kcat constraints applied: {constraints_applied}")
    return constraints_applied > 0

# Apply proper kcat constraints to GECKO models
print("🔧 Fixing kcat constraint implementation...")
proper_constraints_success = add_proper_kcat_constraints(gecko_models, reac_gene, gene_uniprot)

if proper_constraints_success:
    print(f"\n✅ SUCCESS! Proper kcat constraints now applied to gecko_models!")
    print(f"🧬 Metabolic reaction fluxes are now limited by enzyme kinetics!")
else:
    print(f"\n❌ Failed to apply proper kcat constraints.")

🔧 Fixing kcat constraint implementation...
Kcat constraints to be applied:
  Q9FD71: kcat=5.2 s⁻¹, availability=0.0010 mmol/gDW
    → Max flux: 0.005200 mmol/gDW/h
  Q9FD65: kcat=3.8 s⁻¹, availability=0.0010 mmol/gDW
    → Max flux: 0.003800 mmol/gDW/h
  O81954: kcat=12.5 s⁻¹, availability=0.0020 mmol/gDW
    → Max flux: 0.025000 mmol/gDW/h
  O81014: kcat=8.9 s⁻¹, availability=0.0010 mmol/gDW
    → Max flux: 0.008900 mmol/gDW/h
  B4DWP0: kcat=15.3 s⁻¹, availability=0.0010 mmol/gDW
    → Max flux: 0.015300 mmol/gDW/h
  P52020: kcat=7.2 s⁻¹, availability=0.0005 mmol/gDW
    → Max flux: 0.003600 mmol/gDW/h
Applying constraints to model: SQE+MVA_updated
  Processing ALT_MVAS with enzymes: ['Q9FD71']
    ✓ Applied constraint: (0.0, 1000.0) → (0.0, 0.005200000000000001)
      Limiting enzyme: Q9FD71 (max flux: 0.005200)
  Processing ALT_MVAE with enzymes: ['Q9FD65']
    ✓ Applied constraint: (0.0, 1000.0) → (0.0, 0.0038)
      Limiting enzyme: Q9FD65 (max flux: 0.003800)
  Processing DXS wit

### FBA

In [70]:
# Optimize GECKO models with ERGOSTEROLEXCH as objective
import pandas as pd

def optimize_ergosterol_production(gecko_models_dict):
    """
    Optimize ergosterol production for each GECKO model using ERGOSTEROLEXCH as objective
    """
    results = {}
    
    print("=== ERGOSTEROL PRODUCTION OPTIMIZATION ===\n")
    
    for model_name, gecko_model in gecko_models_dict.items():
        print(f"Optimizing model: {model_name}")
        
        try:
            # Check if ERGOSTEROLEXCH reaction exists
            if 'ERGOSTEROLEXCH' not in [r.id for r in gecko_model.reactions]:
                print(f"  ✗ ERGOSTEROLEXCH reaction not found in {model_name}")
                results[model_name] = {
                    'status': 'error',
                    'message': 'ERGOSTEROLEXCH reaction not found',
                    'objective_value': None,
                    'ergosterol_flux': None
                }
                continue
            
            # Get the ERGOSTEROLEXCH reaction
            erg_reaction = gecko_model.reactions.get_by_id('ERGOSTEROLEXCH')
            # print(f"  Found ERGOSTEROLEXCH: {erg_reaction.name}")
            # print(f"  Reaction formula: {erg_reaction.reaction}")
            # print(f"  Current bounds: {erg_reaction.bounds}")
            
            # Set ERGOSTEROLEXCH as the objective (maximize ergosterol production)
            gecko_model.objective = 'ERGOSTEROLEXCH'
            gecko_model.objective.direction = 'max'
            
            # print(f"  Set objective to maximize ERGOSTEROLEXCH")
            
            # Optimize the model
            solution = gecko_model.optimize()
            
            if solution.status == 'optimal':
                ergosterol_flux = solution.fluxes['ERGOSTEROLEXCH']
                # print(f"  ✓ Optimization successful")
                # print(f"  Ergosterol production flux: {ergosterol_flux:.6f}")
                # print(f"  Objective value: {solution.objective_value:.6f}")
                
                # Get some key pathway fluxes for context
                key_reactions = ['SS', 'SMO', 'ERG5', 'ERG3', 'ERG']
                pathway_fluxes = {}
                for rxn_id in key_reactions:
                    if rxn_id in solution.fluxes.index:
                        pathway_fluxes[rxn_id] = solution.fluxes[rxn_id]
                
                results[model_name] = {
                    'status': 'optimal',
                    'objective_value': solution.objective_value,
                    'ergosterol_flux': ergosterol_flux,
                    'pathway_fluxes': pathway_fluxes,
                    'message': 'Successfull'
                }
                
                # if pathway_fluxes:
                #     print(f"  Key sterol pathway fluxes:")
                #     for rxn, flux in pathway_fluxes.items():
                #         print(f"    {rxn}: {flux:.6f}")
                        
            else:
                # print(f"  ✗ Optimization failed: {solution.status}")
                results[model_name] = {
                    'status': solution.status,
                    'objective_value': None,
                    'ergosterol_flux': None,
                    'message': f'Optimization status: {solution.status}'
                }
                
        except Exception as e:
            # print(f"  ✗ Error optimizing {model_name}: {str(e)}")
            results[model_name] = {
                'status': 'error',
                'objective_value': None,
                'ergosterol_flux': None,
                'message': str(e)
            }
        
        print()  # Add spacing between models
    
    return results

# Run the optimization
optimization_results = optimize_ergosterol_production(gecko_models)

# Create a summary table
print("=== OPTIMIZATION RESULTS SUMMARY ===")
summary_data = []
for model_name, result in optimization_results.items():
    summary_data.append({
        'Model': model_name,
        'Status': result['status'],
        'Ergosterol Flux': result['ergosterol_flux'],
        'Objective Value': result['objective_value'],
        'Message': result['message']
    })

summary_df = pd.DataFrame(summary_data)
print(summary_df.to_string(index=False))

# Show pathway flux comparison for successful optimizations
successful_models = {k: v for k, v in optimization_results.items() if v['status'] == 'optimal'}
if successful_models:
    print(f"\n=== STEROL PATHWAY FLUX COMPARISON ===")
    pathway_comparison = []
    for model_name, result in successful_models.items():
        row = {'Model': model_name}
        row.update(result['pathway_fluxes'])
        pathway_comparison.append(row)
    
    if pathway_comparison:
        pathway_df = pd.DataFrame(pathway_comparison)
        print(pathway_df.to_string(index=False))

=== ERGOSTEROL PRODUCTION OPTIMIZATION ===

Optimizing model: SQE+MVA_updated

Optimizing model: SQE_updated

Optimizing model: SQS+MVA_updated

Optimizing model: SQS+SQE+MVA_updated

Optimizing model: SQS+SQE_updated

Optimizing model: SQS_updated

Optimizing model: Wildtype_updated

=== OPTIMIZATION RESULTS SUMMARY ===
              Model  Status  Ergosterol Flux  Objective Value     Message
    SQE+MVA_updated optimal         0.002117         0.002117 Successfull
        SQE_updated optimal         0.001483         0.001483 Successfull
    SQS+MVA_updated optimal         0.002117         0.002117 Successfull
SQS+SQE+MVA_updated optimal         0.002117         0.002117 Successfull
    SQS+SQE_updated optimal         0.001483         0.001483 Successfull
        SQS_updated optimal         0.001483         0.001483 Successfull
   Wildtype_updated optimal         0.001483         0.001483 Successfull

=== STEROL PATHWAY FLUX COMPARISON ===
              Model       SS           SMO   